In [14]:
import pandas as pd
import numpy as np
from time import time
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
#from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_validate, GridSearchCV
from sklearn.metrics import make_scorer, mean_absolute_percentage_error

In [15]:
# Loading the data

df = pd.read_csv('../data/preprocessed/newborntrain_processed.csv', sep=";")
#print(df.head())

# Removing the target variable from the feature

X = df.drop("newborn_weight", axis=1)
Y = df["newborn_weight"]

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=12345)

MLmodel = RandomForestRegressor(n_estimators=100, max_depth=10)
MLmodel.fit(X_train,Y_train)

#Generating Predictions


RandomForestRegressor(max_depth=10)

In [17]:
Y_Pred = MLmodel.predict(X_test)

Mape = mean_absolute_percentage_error(Y_test,Y_Pred)
print(Mape)

0.15711632071528606


## Data Predictn

In [18]:
df_test = pd.read_csv("../data/preprocessed/newborntest_processed.csv", sep=";")
#print(df_test.info)

pred = MLmodel.predict(df_test)
df_test['predicted_weight'] = pred
print(pred.shape)
print(pred)

df_test.to_csv('..\data\Result\RandomForestPred.csv')

(599561,)
[3304.0421927  3323.59606465 3106.21339174 ... 3325.07555037 3515.67726969
 3144.05924391]


## Grid Search

In [19]:
def gs_to_df(grid_search):
    df = pd.DataFrame()
    for param in param_grid[0].keys():
        df[param] = 0
    df["mean_train_score"] = df["std_train_score"] = df["mean_test_score"] = df["std_test_score"] = 0
    for i in range(len(grid_search.cv_results_['params'])):
        new_row = grid_search.cv_results_['params'][i] | {"mean_train_score": grid_search.cv_results_['mean_train_score'][i],
                                                            "std_train_score": grid_search.cv_results_['std_train_score'][i],
                                                            "mean_test_score": grid_search.cv_results_['mean_test_score'][i],
                                                            "std_test_score": grid_search.cv_results_['std_test_score'][i]}
        df.loc[len(df)] = new_row

    return df

In [20]:
start_time = time()
# Grid 
param_grid = [{
    'n_estimators': [50, 100, 200],  # Example values for n_estimators
    'max_depth': [5, 10]  # Example values for max_depth
}]

model = RandomForestRegressor()

# Cross Validation
rkf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=12345)
# Grid Search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring="neg_mean_absolute_percentage_error", 
                            n_jobs=-1, cv=rkf, return_train_score=True)
grid_search.fit(X,Y)
print("--- %s seconds ---" % (time() - start_time))

--- 54452.195158958435 seconds ---


In [24]:
df = gs_to_df(grid_search=grid_search)
df.sort_values(ascending=False,by="mean_test_score").to_csv('./Validation/RandomForest_metrics_2_5_cv.csv', index=False,sep=';')
df.sort_values(ascending=False,by="mean_test_score")
#print(df_sorted)

,n_estimators,max_depth,mean_train_score,std_train_score,mean_test_score,std_test_score
5,200,10,-0.156120,0.000135,-0.156878,0.000573
4,100,10,-0.156127,0.000133,-0.156885,0.000573
3,50,10,-0.156136,0.000140,-0.156892,0.000563
2,200,5,-0.161008,0.000159,-0.161043,0.000566
1,100,5,-0.161010,0.000155,-0.161045,0.000571
0,50,5,-0.161011,0.000161,-0.161047,0.000565
